# Arweaveに画像とオフチェーンメタデータをアップロードする

Pythonを利用してArweaveに画像とオフチェーンメタデータをアップロードをしていく

- [Google Colab で実行するためのリンク](https://colab.research.google.com/github/regonn/Solana-NFT/blob/master/SolanaNFT_03.ipynb)

- [Softgateさんの NFT 技術基礎論](https://quirky-caution-3c5.notion.site/NFT-507d8038e2cd46de831e42ba2b4c0a90)

- [Arweave](https://www.arweave.org/)

## AR ウォレット作成
- 初回ウォレット作成時に少額の AR をくれる
  - https://faucet.arweave.net/
- 今回は作成時に作られる json ファイルが必要
  - ※ウォレットキーファイルのためアップロードしたくない場合はローカルのPython環境での実行をおすすめします

## 環境構築

In [ ]:
!pip install arweave-python-client -U

## 画像生成

In [ ]:
import random
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

In [ ]:
# 画像生成用シード値
seed_number = 46
# 画像上のオブジェクトサイズ
OBJ_SIZE = 10
# 画像に出現するオブジェクト数
OBJ_NUMBERS = 100

In [ ]:
def generate_random_image(seed):
    IMG_SIZE = 320
    random.seed(seed)
    im = Image.new("RGB", (IMG_SIZE, IMG_SIZE))
    draw = ImageDraw.Draw(im)
    for i in range(OBJ_NUMBERS):
        pos_x = int(random.random() * IMG_SIZE)
        pos_y = int(random.random() * IMG_SIZE)
        r = random.randint(0,256)
        g = random.randint(0,256)
        b = random.randint(0,256)
        draw.ellipse((pos_x - OBJ_SIZE, 
                      pos_y - OBJ_SIZE, 
                      pos_x + OBJ_SIZE, 
                      pos_y + OBJ_SIZE), 
                      (r,g,b))
    imshow(im)
    im.save('./bubbles.png')

In [ ]:
# seed値を変えると異なる画像が生成される
generate_random_image(seed_number)

## 画像をArweaveへアップロード
- アップロード費用計算サイト
  - https://arweavefees.com/
- 今回の画像は9KB程度で見積もりは
  - 0.000005546966 AR (0.00023 USD)

In [ ]:
import arweave
import json

In [ ]:
# ※ウォレットキーファイルのアップロードには注意してください
wallet_file_path = "./arweave-keyfile.json"
wallet = arweave.Wallet(wallet_file_path)

In [ ]:
balance = wallet.balance

In [ ]:
balance

In [ ]:
# コード実行で実際にトランザクション(アップロード処理)が実行されるので注意
with open('./bubbles.png', 'rb') as img:
    img_data = img.read()

    transaction = arweave.Transaction(wallet, data=img_data)
    transaction.add_tag("Content-Type", "image/png")
    transaction.sign()
    transaction.send()


In [ ]:
transaction_data = transaction.to_dict()

In [ ]:
transaction_data

In [ ]:
image_url = f"https://www.arweave.net/{transaction_data['id']}?ext=png"

In [ ]:
image_url

## オフチェーン用JSONメタデータ作成
- Metaplex で指定されている JSON 構造
  - https://docs.metaplex.com/nft-standard#uri-json-schema

In [ ]:
name = f'Bubbles #{seed_number}'
metadata = {
    'name': name, # 名前
    'symbol': "", # シンボル名(特に必須ではなさそう)
    'description': "What a beautiful bubbles!", # 説明
    'seller_fee_basis_points': 500, # セカンダリーマーケットで指定した creators に渡るロイヤリティ (0-10000) 500で5%
    'external_url': "https://twitter.com/regonn_haizine", # 本来はNFTのページとかを貼る用の外部URL
    'attributes': [ # アイテムに関する情報、opensea の形式に則っているっぽい https://docs.opensea.io/docs/metadata-standards#attributes value は数字か文字列
        {
            'trait_type': "name",
            'value': name
        },
        {
            'trait_type': "obj_size",
            'value': OBJ_SIZE
        },
        {
            'trait_type': "obj_numbers",
            'value': OBJ_NUMBERS
        },
    ],
    'collection': { # コレクション名(よくあるNFTの偽物ってここの情報を本物と同じに合わせているだけなのかも?)
        'name': "Bubbles",
        'family': "NFT Study",
    },
    'properties': { # ユーザに表示する情報をまとめたもの、表示する画像等
        'files': [
            {
                'uri': image_url,
                'type': "image/png",
            },
        ],
        'category': "image", # "image", "video", "audio", "vr" のカテゴリが存在する
        'creators': [
            {
                'address': "A8r5gPBeUHbguZ6mKGB1zzbKhMHtfQdWx6YqXQ94Ujjd",
                'share': 100,
            },
        ],
    },
    'image': image_url,
}

In [ ]:
metadata

In [ ]:
json_str = json.dumps(metadata)

In [ ]:
json_str

In [ ]:
# コード実行で実際にトランザクション(アップロード処理)が実行されるので注意
metadata_transaction = arweave.Transaction(wallet, data=json_str)
metadata_transaction.add_tag("Content-Type", "application/json")
metadata_transaction.sign()
metadata_transaction.send()

In [ ]:
metadata_transaction_data = metadata_transaction.to_dict()

In [ ]:
metadata_transaction_data

In [ ]:
json_url = f"https://www.arweave.net/{metadata_transaction_data['id']}"

In [ ]:
json_url

## arql を利用して過去のトランザクション情報を取得

In [ ]:
from arweave.arweave_lib import arql

In [ ]:
transaction_ids = arql(
    wallet,
    {
        "op": "equals",
        "expr1": "from",
        "expr2": "ZF8XWGJFSj7bPlJCmXaJOIhianqkiUksMRqwAjz2kU8"
    })

In [ ]:
transaction_ids

In [ ]:
tx = arweave.Transaction(wallet, id=transaction_ids[0])
tx.get_transaction()

In [ ]:
tx.get_data()
tx.data